In [4]:
%%classpath add mvn
org.clustering4ever core_2.11 0.9.8
org.clustering4ever clusteringspark_2.11 0.9.8

In [5]:
%%classpath add mvn
org.apache.spark spark-core_2.12 2.4.5
org.apache.spark spark-streaming_2.12 2.4.5

In [47]:
import org.clustering4ever.spark.clustering.s2gstream.S2gstream
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.streaming.{Milliseconds, StreamingContext}
import org.apache.spark.streaming.dstream.DStream
import org.apache.log4j.Logger
import org.apache.log4j.Level


import org.clustering4ever.spark.clustering.s2gstream.S2gstream
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.streaming.{Milliseconds, StreamingContext}
import org.apache.spark.streaming.dstream.DStream
import org.apache.log4j.Logger
import org.apache.log4j.Level


In [21]:
val master = "local[2]"
val dirData = "streams"
val dirSortie = "results"
val path =  "C://Users/ATTAOUI/Documents/SS_GStream/resources/DS1.txt"
val separator = ","
val decayFactor = 0.99
val lambdaAge = 1.2
val nbNodesToAdd = 3
val nbWind = 60
var batchNumber = 1
var batchIndex = 0
val batchSize = 100


100

In [48]:
val sparkConf = new SparkConf().setAppName(this.getClass().getName())
  sparkConf.setMaster(master).set("spark.driver.allowMultipleContexts", "true")
val sc = new SparkContext(sparkConf)

org.apache.spark.SparkContext@73db797f

In [50]:
// Create a DStreams that reads batch files from dirData
@transient def createDstream(ssc : StreamingContext, file: String, separator: String): DStream[Array[Double]] ={
  val stream = ssc.textFileStream(file).map(x => x.split(separator).map(_.toDouble))
  return stream
}

createDstream: (ssc: org.apache.spark.streaming.StreamingContext, file: String, separator: String)org.apache.spark.streaming.dstream.DStream[Array[Double]]


In [51]:
val interval = 100
val batchInterval = Milliseconds(interval)
val ssc = new StreamingContext(sc, batchInterval)
val stream = createDstream(ssc, dirData, separator)
stream

org.apache.spark.streaming.dstream.MappedDStream@577578cd

In [52]:
val data = sc.textFile(path).map(x => x.split(',').map(_.toDouble))
val v = data.take(1)(0).length
val data2 = data.collect()
var labels =  data.map(x => x(v - 1).toInt).collect()
val points2 = sc.parallelize(data2.slice(0, 2)) //sc.textFile("resources/nodes2.txt").map(x => x.split(separator).map(_.toDouble))

ParallelCollectionRDD[4] at parallelize at <console>:122

In [55]:
import org.clustering4ever.spark.streamclustering.streamData

val labId = 2 //TODO: change -1 to -2 when you add the id to the file (last column) //-2 because the last 2 columns represent label & id
val dim = points2.take(1)(0).size - labId
var gstream = new S2gstream()
    .setDecayFactor(decayFactor)
    .setLambdaAge(lambdaAge)
    .setMaxInsert(nbNodesToAdd)
  // converting each point into an object
val dstreamObj = stream.map( e =>
    gstream.model.pointToObjet(e, dim, labId)
)
var stream2 = new Array[streamData](0)
while(batchNumber <= (data2.length / batchSize) ){
    val batch = data2.slice(batchIndex, batchIndex + batchSize)
    stream2 :+= new streamData(batch.map(a => gstream.model.pointToObjet(a, dim, labId)))
    batchIndex += batchSize
    batchNumber += 1
}
val streamRDD = stream2.map(x => sc.parallelize(x.stream))
dstreamObj.cache() //TODO: to save in memory


null

In [56]:
// initialization of the model by creating a graph of two nodes (the first 2 data-points)
gstream.initModelObj(points2, dim)

// training on the model
val trainedModel = gstream.trainOnObj(dstreamObj, gstream, dirSortie + "/dataset" + nbNodesToAdd, dim, nbWind, streamRDD, labels, data2)


node: 1 -> 122.0, 199.0
node: 2 -> 243.0, 434.0